# Analyse Eye-Trackingdaten Sarah/Kristin
# 25.07.2018
rm(list=ls())

library(data.table)
library(dplyr)

setwd("C:/Users/datenverlust/Desktop/Studium/Master/Masterarbeit/Auswertung_Experiment")

#### Datenmanipulation ####
#daten <- read.table("D1_raw.txt", header = T, sep = "\t")
#data.table liest die Daten deutlich schneller ein als read.table
#verlangt aber eine andere Syntax zum LÃ¶schen von Spalten und Zeilen

#Daten einlesen
daten <- fread("Raw Data_farben.txt", sep = "\t")
AOI_values <- read.table("AOI.txt", header = T)
Onset <- read.table("Onset.txt", header = T)

#Variablennamen definieren
daten <- daten[,c("RecordingTime [ms]", "Trial", "Participant", "Stimulus", "Tracking Ratio [%]", "Category Group",
                     "Category Right", "Point of Regard Right X [px]", "Point of Regard Right Y [px]","AOI Name Right")]
names(daten)[names(daten) == "AOI Name Right"] <- "AOI_right"
names(daten)[names(daten) == "Tracking Ratio [%]"] <- "TrackingRatio"
names(daten)[names(daten) == "Category Group"] <- "CategoryGroup"
names(daten)[names(daten) == "Category Right"] <- "CategoryRight"
names(daten)[names(daten) == "RecordingTime [ms]"] <- "RecordingTime"
names(daten)[names(daten) == "Point of Regard Right X [px]"] <- "PoRRightX"
names(daten)[names(daten) == "Point of Regard Right Y [px]"] <- "PoRRightY"

# Datenbereinigung (Stimulus)
daten_w_validation <- (daten$Stimulus != "Validation.jpg")
daten_wo_validation <- daten[daten_w_validation,]
daten_w_filler  <- substring(daten_wo_validation$Stimulus, 1, nchar("filler")) == "filler"
daten_wo_filler <- daten_wo_validation[!daten_w_filler,]
daten_w_rtf <- (daten_wo_filler$Stimulus != "RichText.rtf")
daten_wo_rtf <- daten_wo_filler[daten_w_rtf,]
daten_w_preview <- daten_wo_rtf[!grep("preview", daten_wo_rtf$Stimulus, fixed = TRUE)]
daten_w_preview <- daten_w_preview[!grep("peview", daten_w_preview$Stimulus, fixed = TRUE)]
daten <- daten_w_preview

# Einfügen neuer Spalten für Item, Bedingung und AOI
daten$Item <- substring(daten$Stimulus, 5,7)
daten$Condition <- substring(daten$Stimulus, 9,9)

#Trialbeginn rausschreiben
trialstart <- daten[CategoryRight == "Separator"]
#einen Spaltennamen definieren
colnames(trialstart)[colnames(trialstart) == "RecordingTime"] <- "trialstart"
#Variablen definieren, die man in der data.table trialstart behalten will
keep <- c("Participant", "Trial", "trialstart")
#ueberfluessige Variablen lÃ¶schen
trialstart <- trialstart[, keep, with = FALSE]
#Trialstart an data.table daten mergen
daten <- merge(daten, trialstart, by = c("Participant", "Trial"))
rm(trialstart, keep)
#Zeit berechnen
daten$time_step <- daten$RecordingTime - daten$trialstart

daten$Onset_Nomen <- ifelse(daten$Item == Onset$Item & daten$Condition == Onset$Condition,
         +        Onset$Nomen_Onset,
         +        99)

daten$Onset_Adj <- ifelse(daten$Item == Onset$Item & daten$Condition == Onset$Condition,
         +         Onset$Adjektiv_Onset1,
         +        99)

# Trackloss auslesen (CategoryRight)
daten$trackloss <- ifelse(daten$CategoryRight == "-" | daten$CategoryRight == "Blink",
                          yes = 1,
                          no = 0)


##https://stackoverflow.com/questions/14102498/merge-dataframes-different-lengths#14102598
## 

write.table(daten,"preprocessed_w.txt",sep="\t",row.names=FALSE)

# Faktorisierunge der Spalten
daten$Item <- as.factor(daten$Item)
daten$RecordingTime <- as.factor(daten$RecordingTime)
daten$Trial <- as.factor(daten$Trial)
daten$Participant <- as.factor(daten$Participant)
daten$Stimulus <- as.factor(daten$Stimulus)
daten$TrackingRatio <- as.factor(daten$TrackingRatio)
daten$CategoryGroup <- as.factor(daten$CategoryGroup)
daten$CategoryRight <- as.factor(daten$CategoryRight)
daten$AOI_right <- as.factor(daten$AOI_right)
daten$Condition <- as.factor(daten$Condition)
daten$trackloss <- as.factor(daten$trackloss)



# Spalte mit Zeilennummern wird eingefügt
daten$column_nr <- 1:nrow(daten)

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from collections import defaultdict

In [2]:
data = pd.read_csv('Raw Data_farben.txt', delimiter="\t", header=0)

/Users/kristinhaake2/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.keys()

Index(['RecordingTime [ms]', 'Time of Day [h:m:s:ms]', 'Trial', 'Stimulus',
       'audio playback', 'Export Start Trial Time [ms]',
       'Export End Trial Time [ms]', 'Participant', 'Erstsprache',
       'Probandennummer', 'Alter', 'Tracking Ratio [%]', 'Category Group',
       'Category Right', 'Index Right', 'Point of Regard Right X [px]',
       'Point of Regard Right Y [px]', 'AOI Name Right'],
      dtype='object')

In [4]:
data

,RecordingTime [ms],Time of Day [h:m:s:ms],Trial,Stimulus,audio playback,Export Start Trial Time [ms],Export End Trial Time [ms],Participant,Erstsprache,Probandennummer,Alter,Tracking Ratio [%],Category Group,Category Right,Index Right,Point of Regard Right X [px],Point of Regard Right Y [px],AOI Name Right
0,12222998.9,14:14:55:659,Trial001,Validation.jpg,Stop,0.0,9664.0,P1,Deutsch,1,27,100.0,Information,Separator,1,-,-,-
1,12223000.2,14:14:55:660,Trial001,Validation.jpg,Stop,0.0,9664.0,P1,Deutsch,1,27,100.0,Eye,Fixation,1,906.3,234.7,10730
2,12223004.2,14:14:55:664,Trial001,Validation.jpg,Stop,0.0,9664.0,P1,Deutsch,1,27,100.0,Eye,Fixation,1,906.3,237.8,10730
3,12223008.1,14:14:55:668,Trial001,Validation.jpg,Stop,0.0,9664.0,P1,Deutsch,1,27,100.0,Eye,Fixation,1,906.6,237.4,10730
4,12223012.0,14:14:55:672,Trial001,Validation.jpg,Stop,0.0,9664.0,P1,Deutsch,1,27,100.0,Eye,Fixation,1,909.2,237.5,10730
5,12223015.9,14:14:55:676,Trial001,Validation.jpg,Stop,0.0,9664.0,P1,Deutsch,1,27,100.0,Eye,Fixation,1,910.3,237.1,10730
6,12223019.9,14:14:55:680,Trial001,Validation.jpg,Stop,0.0,9664.0,P1,Deutsch,1,27,100.0,Eye,Fixation,1,909.8,238.4,10730
7,12223023.8,14:14:55:684,Trial001,Validation.jpg,Stop,0.0,9664.0,P1,Deutsch,1,27,100.0,Eye,Fixation,1,909.0,240.6,10730
8,12223027.7,14:14:55:688,Trial001,Validation.jpg,Stop,0.0,9664.0,P1,Deutsch,1,27,100.0,Eye,Fixation,1,908.7,242.0,10730
9,12223031.7,14:14:55:692,Trial001,Validation.jpg,Stop,0.0,9664.0,P1,Deutsch,1,27,100.0,Eye,Fixation,1,907.9,245.2,10730


In [4]:
probanden = data[['Participant', 'Erstsprache', 'Alter']]

In [5]:
probanden

,Participant,Erstsprache,Alter
0,P1,Deutsch,27
1,P1,Deutsch,27
2,P1,Deutsch,27
3,P1,Deutsch,27
4,P1,Deutsch,27
5,P1,Deutsch,27
6,P1,Deutsch,27
7,P1,Deutsch,27
8,P1,Deutsch,27
9,P1,Deutsch,27


In [5]:
setting_info = set()
for ix in range(len(probanden['Alter'])):
    setting_info.add(probanden['Alter'][ix])

NameError: name 'probanden' is not defined

In [7]:
print(setting_info)

{33, 34, 19, 21, 22, 23, 24, 25, 26, 27, 28}


In [3]:
data['RecTime']= data['RecordingTime [ms]']
data['TimeofDay'] = data['Time of Day [h:m:s:ms]']
data['TrackingRatio'] = data['Tracking Ratio [%]']
data['CategoryGroup'] = data['Category Group']
data['CategoryRight'] = data['Category Right']
data['PointRRX'] = data['Point of Regard Right X [px]']
data['PointRRY'] = data['Point of Regard Right Y [px]']

In [4]:
data = data[["RecTime", "Trial", "Participant", "Stimulus", "TrackingRatio", "CategoryGroup", "CategoryRight", "PointRRX", "PointRRY"]]

In [5]:
validation_data = data[data.Stimulus == "Validation.jpg"]
changed_data = data.drop(validation_data.index, axis=0)
richtext_data = changed_data[changed_data.Stimulus == "RichText.rtf"]
changed_data = changed_data.drop(richtext_data.index, axis=0)
person_data = changed_data[changed_data.Participant == "P19"]
changed_data = changed_data.drop(person_data.index, axis=0)
changed_data.index = range(len(changed_data))


In [6]:
indexing = []
for ix in range(len(changed_data['Stimulus'])):
    if changed_data['Stimulus'][ix].startswith('filler'):
        indexing.append(ix)
    elif changed_data['Stimulus'][ix].endswith('preview'):
        indexing.append(ix)
    elif changed_data['Stimulus'][ix].endswith('previews'):
        indexing.append(ix)
    elif changed_data['Stimulus'][ix].endswith('peview'):
        indexing.append(ix)

In [7]:
changed_data= changed_data.drop(changed_data.index[[indexing]])

/Users/kristinhaake2/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:2095: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [8]:
changed_data.index = range(len(changed_data))

In [9]:
click_data = changed_data[changed_data.CategoryRight == "Left Click"]
changed_data = changed_data.drop(click_data.index, axis=0)
changed_data.index = range(len(changed_data))

In [10]:
item = []
condi = []
for ix in range(len(changed_data['Stimulus'])):
    condi.append(int(changed_data['Stimulus'][ix].split('_')[1]))
    item.append(int(changed_data['Stimulus'][ix].split('m')[1].split('_')[0]))

changed_data['Item'] = item
changed_data['Condition'] = condi

In [11]:
trackloss = []
for ix in range(len(changed_data['CategoryRight'])):
    if changed_data['CategoryRight'][ix] == '-' or changed_data['CategoryRight'][ix] == 'Blink':
        trackloss.append(1)
    else:
        trackloss.append(0)

changed_data['trackloss'] = trackloss

In [12]:
trialstart = []
time_step = []
for ix in range(len(changed_data['RecTime'])):
    if changed_data['CategoryRight'][ix] == 'Separator':
        start = changed_data['RecTime'][ix]
        trialstart.append(start)  
        time_step.append('nan')
    elif changed_data['CategoryRight'][ix-1] == 'Separator':
        time_step.append(changed_data['RecTime'][ix]-start)
        trialstart.append(start)  

    else:
        trialstart.append(start)  
        time_step.append(changed_data['RecTime'][ix]-start)

In [13]:
changed_data['trialstart'] = trialstart
changed_data['time_step'] = time_step

In [14]:
separator_data = changed_data[changed_data.CategoryRight == "Separator"]
changed_data = changed_data.drop(separator_data.index, axis=0)
changed_data.index = range(len(changed_data))

In [15]:
data = changed_data[["Item", "Condition", "Trial", "Participant", "Stimulus", "CategoryGroup", "CategoryRight", "PointRRX", "PointRRY", "time_step","trackloss"]]

In [19]:
data

,Item,Condition,Trial,Participant,Stimulus,CategoryGroup,CategoryRight,PointRRX,PointRRY,time_step,trackloss
0,121,3,Trial012,P1,item121_3,Eye,Fixation,525.3,920.6,3.5,0
1,121,3,Trial012,P1,item121_3,Eye,Fixation,524.1,921.7,7.5,0
2,121,3,Trial012,P1,item121_3,Eye,Fixation,524.3,920.9,11.5,0
3,121,3,Trial012,P1,item121_3,Eye,Fixation,524.2,918.5,15.5,0
4,121,3,Trial012,P1,item121_3,Eye,Fixation,522.5,919.3,19.5,0
5,121,3,Trial012,P1,item121_3,Eye,Fixation,523.5,920.1,23.5,0
6,121,3,Trial012,P1,item121_3,Eye,Fixation,523.5,919.2,27.5,0
7,121,3,Trial012,P1,item121_3,Eye,Fixation,525.1,918.3,31.5,0
8,121,3,Trial012,P1,item121_3,Eye,Fixation,527.1,918.9,35.5,0
9,121,3,Trial012,P1,item121_3,Eye,Fixation,528.5,918.6,39.5,0


In [16]:
Onset = pd.read_csv('Onset.txt', delimiter="\t",header=0)

In [19]:
Onset.keys()

Index(['Item', 'Condition', 'Adjektiv_Onset1', 'Nomen_Onset'], dtype='object')

In [17]:
data = pd.merge(data,Onset)

In [22]:
data

,Item,Condition,Trial,Participant,Stimulus,CategoryGroup,CategoryRight,PointRRX,PointRRY,time_step,trackloss,Adjektiv_Onset1,Nomen_Onset
0,121,3,Trial012,P1,item121_3,Eye,Fixation,525.3,920.6,3.5,0,0.575,1.52
1,121,3,Trial012,P1,item121_3,Eye,Fixation,524.1,921.7,7.5,0,0.575,1.52
2,121,3,Trial012,P1,item121_3,Eye,Fixation,524.3,920.9,11.5,0,0.575,1.52
3,121,3,Trial012,P1,item121_3,Eye,Fixation,524.2,918.5,15.5,0,0.575,1.52
4,121,3,Trial012,P1,item121_3,Eye,Fixation,522.5,919.3,19.5,0,0.575,1.52
5,121,3,Trial012,P1,item121_3,Eye,Fixation,523.5,920.1,23.5,0,0.575,1.52
6,121,3,Trial012,P1,item121_3,Eye,Fixation,523.5,919.2,27.5,0,0.575,1.52
7,121,3,Trial012,P1,item121_3,Eye,Fixation,525.1,918.3,31.5,0,0.575,1.52
8,121,3,Trial012,P1,item121_3,Eye,Fixation,527.1,918.9,35.5,0,0.575,1.52
9,121,3,Trial012,P1,item121_3,Eye,Fixation,528.5,918.6,39.5,0,0.575,1.52


In [18]:
adj_onset = []
#adj_onset2 = []
noun_onset = []
for ix in range(len(data['Adjektiv_Onset1'])):
    adj_onset.append(data['Adjektiv_Onset1'][ix]*1000)
    #adj_onset2.append(data['Adjektiv_Onset2'][ix]*1000)
    noun_onset.append(data['Nomen_Onset'][ix]*1000)
    
data['Adj_Onset'] = adj_onset
#data['Adj_Onset2'] = adj_onset2
data['Noun_Onset'] = noun_onset

In [19]:
adj_time = []
#adj2_time = []
noun_time = []
for ix in range(len(data['time_step'])):
    adj_time.append(data['time_step'][ix]-int(data['Adj_Onset'][ix]))
    #adj2_time.append(data['time_step'][ix]-int(data['Adj_Onset2'][ix]))
    noun_time.append(data['time_step'][ix]-int(data['Noun_Onset'][ix]))

data['adjtime'] = adj_time
#data['adj2time'] = adj2_time
data['nountime'] = noun_time

In [24]:
data['nountime']

0        -1516.5
1        -1512.5
2        -1508.5
3        -1504.5
4        -1500.5
5        -1496.5
6        -1492.5
7        -1488.5
8        -1484.5
9        -1480.5
10       -1476.5
11       -1472.5
12       -1468.5
13       -1464.5
14       -1460.5
15       -1456.5
16       -1452.5
17       -1448.5
18       -1444.5
19       -1440.5
20       -1436.5
21       -1432.5
22       -1428.5
23       -1424.5
24       -1420.5
25       -1416.5
26       -1412.5
27       -1408.5
28       -1404.5
29       -1400.5
           ...  
748562    1699.6
748563    1703.6
748564    1707.6
748565    1711.6
748566    1715.6
748567    1719.6
748568    1723.6
748569    1727.6
748570    1731.6
748571    1735.6
748572    1739.6
748573    1743.6
748574    1747.6
748575    1751.6
748576    1755.6
748577    1759.6
748578    1763.6
748579    1767.6
748580    1771.6
748581    1775.6
748582    1779.6
748583    1783.6
748584    1787.6
748585    1791.6
748586    1795.6
748587    1799.6
748588    1803.6
748589    1807

In [20]:
aoi = pd.read_csv('AOI.txt', delimiter="\t", header=0)
#aoi

In [26]:
data

,Item,Condition,Trial,Participant,Stimulus,CategoryGroup,CategoryRight,PointRRX,PointRRY,time_step,trackloss,Adjektiv_Onset1,Nomen_Onset,Adj_Onset,Noun_Onset,adjtime,nountime
0,121,3,Trial012,P1,item121_3,Eye,Fixation,525.3,920.6,3.5,0,0.575,1.52,575.0,1520.0,-571.5,-1516.5
1,121,3,Trial012,P1,item121_3,Eye,Fixation,524.1,921.7,7.5,0,0.575,1.52,575.0,1520.0,-567.5,-1512.5
2,121,3,Trial012,P1,item121_3,Eye,Fixation,524.3,920.9,11.5,0,0.575,1.52,575.0,1520.0,-563.5,-1508.5
3,121,3,Trial012,P1,item121_3,Eye,Fixation,524.2,918.5,15.5,0,0.575,1.52,575.0,1520.0,-559.5,-1504.5
4,121,3,Trial012,P1,item121_3,Eye,Fixation,522.5,919.3,19.5,0,0.575,1.52,575.0,1520.0,-555.5,-1500.5
5,121,3,Trial012,P1,item121_3,Eye,Fixation,523.5,920.1,23.5,0,0.575,1.52,575.0,1520.0,-551.5,-1496.5
6,121,3,Trial012,P1,item121_3,Eye,Fixation,523.5,919.2,27.5,0,0.575,1.52,575.0,1520.0,-547.5,-1492.5
7,121,3,Trial012,P1,item121_3,Eye,Fixation,525.1,918.3,31.5,0,0.575,1.52,575.0,1520.0,-543.5,-1488.5
8,121,3,Trial012,P1,item121_3,Eye,Fixation,527.1,918.9,35.5,0,0.575,1.52,575.0,1520.0,-539.5,-1484.5
9,121,3,Trial012,P1,item121_3,Eye,Fixation,528.5,918.6,39.5,0,0.575,1.52,575.0,1520.0,-535.5,-1480.5


In [21]:
data = pd.merge(data,aoi)

In [22]:
target = []
competitor = []
distractor = []

for ix in range(len(data['PointRRX'])):
    if data['PointRRX'][ix] == '-' or data['PointRRY'][ix] == '-':
        competitor.append(0)
        distractor.append(0)
        target.append(0)
    elif float(data['PointRRX'][ix]) >= float(data['target_xmin'][ix]) and \
    float(data['PointRRX'][ix]) <= float(data['target_xmax'][ix]) and \
    float(data['PointRRY'][ix]) >= float(data['target_ymin'][ix]) and \
    float(data['PointRRY'][ix]) <= float(data['target_ymax'][ix]):
        target.append(1)
        competitor.append(0)
        distractor.append(0)
    elif float(data['PointRRX'][ix]) >= float(data['competitor_xmin'][ix]) and \
    float(data['PointRRX'][ix]) <= float(data['competitor_xmax'][ix]) and \
    float(data['PointRRY'][ix]) >= float(data['competitor_ymin'][ix]) and \
    float(data['PointRRY'][ix]) <= float(data['competitor_ymax'][ix]):
        competitor.append(1)
        distractor.append(0)
        target.append(0)
    elif float(data['PointRRX'][ix]) >= float(data['distractor_xmin'][ix]) and \
    float(data['PointRRX'][ix]) <= float(data['distractor_xmax'][ix]) and \
    float(data['PointRRY'][ix]) >= float(data['distractor_ymin'][ix]) and \
    float(data['PointRRY'][ix]) <= float(data['distractor_ymax'][ix]):
        distractor.append(1)
        competitor.append(0)
        target.append(0)
    else:
        competitor.append(0)
        distractor.append(0)
        target.append(0)
        
data['target'] = target       
data['competitor'] = competitor
data['distractor'] = distractor

In [28]:
#index = [ix+1 for ix in range(len(data['Item']))]
#data['Row_no'] = index

In [29]:
data

,Item,Condition,Trial,Participant,Stimulus,CategoryGroup,CategoryRight,PointRRX,PointRRY,time_step,...,competitor_ymin,competitor_xmax,competitor_ymax,distractor_xmin,distractor_ymin,distractor_xmax,distractor_ymax,target,competitor,distractor
0,121,3,Trial012,P1,item121_3,Eye,Fixation,525.3,920.6,3.5,...,65,1215,265,363,823,686,1003,0,0,1
1,121,3,Trial012,P1,item121_3,Eye,Fixation,524.1,921.7,7.5,...,65,1215,265,363,823,686,1003,0,0,1
2,121,3,Trial012,P1,item121_3,Eye,Fixation,524.3,920.9,11.5,...,65,1215,265,363,823,686,1003,0,0,1
3,121,3,Trial012,P1,item121_3,Eye,Fixation,524.2,918.5,15.5,...,65,1215,265,363,823,686,1003,0,0,1
4,121,3,Trial012,P1,item121_3,Eye,Fixation,522.5,919.3,19.5,...,65,1215,265,363,823,686,1003,0,0,1
5,121,3,Trial012,P1,item121_3,Eye,Fixation,523.5,920.1,23.5,...,65,1215,265,363,823,686,1003,0,0,1
6,121,3,Trial012,P1,item121_3,Eye,Fixation,523.5,919.2,27.5,...,65,1215,265,363,823,686,1003,0,0,1
7,121,3,Trial012,P1,item121_3,Eye,Fixation,525.1,918.3,31.5,...,65,1215,265,363,823,686,1003,0,0,1
8,121,3,Trial012,P1,item121_3,Eye,Fixation,527.1,918.9,35.5,...,65,1215,265,363,823,686,1003,0,0,1
9,121,3,Trial012,P1,item121_3,Eye,Fixation,528.5,918.6,39.5,...,65,1215,265,363,823,686,1003,0,0,1


In [23]:
# ohne: 'Row_no'
data = data[['Stimulus','Participant','Trial','time_step', 'target', 'competitor', 'distractor', 'trackloss', 'Item', 'Condition', 'Adjektiv_Onset1',  'Nomen_Onset','adjtime',  'nountime']]#,'Target','Competitor','Distractor']]

In [33]:
data = data[['Row_no','Stimulus','Participant','Trial','time_step', 'target', 'competitor', 'distractor', 'trackloss', 'Item', 'Condition', 'Adjektiv_Onset1',  'Nomen_Onset','adjtime',  'nountime']]
data.to_csv(r'data.txt', header=1, index=None, sep='\t')

In [31]:
click = pd.read_csv('click_data_separate.txt', delimiter="\t",header=0)

In [32]:
data = pd.merge(data,click)

In [33]:
index = [ix+1 for ix in range(len(data['Item']))]
data['Row_no'] = index

In [34]:
final_data = data[['Row_no','Stimulus','Participant','Trial','time_step', 'target', 'competitor', 'distractor', 'trackloss', 'Item', 'Condition', 'Adjektiv_Onset1',  'Nomen_Onset','adjtime',  'nountime','Target','Competitor','Distractor']]

In [35]:
final_data

,Row_no,Stimulus,Participant,Trial,time_step,target,competitor,distractor,trackloss,Item,Condition,Adjektiv_Onset1,Nomen_Onset,adjtime,nountime,Target,Competitor,Distractor
0,1,item121_3,P1,Trial012,3.5,0,0,1,0,121,3,0.575,1.52,-571.5,-1516.5,1,0,0
1,2,item121_3,P1,Trial012,7.5,0,0,1,0,121,3,0.575,1.52,-567.5,-1512.5,1,0,0
2,3,item121_3,P1,Trial012,11.5,0,0,1,0,121,3,0.575,1.52,-563.5,-1508.5,1,0,0
3,4,item121_3,P1,Trial012,15.5,0,0,1,0,121,3,0.575,1.52,-559.5,-1504.5,1,0,0
4,5,item121_3,P1,Trial012,19.5,0,0,1,0,121,3,0.575,1.52,-555.5,-1500.5,1,0,0
5,6,item121_3,P1,Trial012,23.5,0,0,1,0,121,3,0.575,1.52,-551.5,-1496.5,1,0,0
6,7,item121_3,P1,Trial012,27.5,0,0,1,0,121,3,0.575,1.52,-547.5,-1492.5,1,0,0
7,8,item121_3,P1,Trial012,31.5,0,0,1,0,121,3,0.575,1.52,-543.5,-1488.5,1,0,0
8,9,item121_3,P1,Trial012,35.5,0,0,1,0,121,3,0.575,1.52,-539.5,-1484.5,1,0,0
9,10,item121_3,P1,Trial012,39.5,0,0,1,0,121,3,0.575,1.52,-535.5,-1480.5,1,0,0


In [36]:
#export table
final_data.to_csv(r'final_data_clicks.txt', header=1, index=None, sep='\t')

In [24]:
click_obj = pd.read_csv('click_data_w_objectname.txt', delimiter="\t",header=0)
click_obj

,Participant,Item,Condition,Klick on
0,P9,116,3,target
1,P21,116,3,target
2,P12,116,3,target
3,P18,116,3,target
4,P15,116,3,target
5,P3,116,3,target
6,P6,116,3,target
7,P3,117,1,target
8,P9,117,1,target
9,P6,117,1,target


In [25]:
click_obj['Klick']= click_obj['Klick on']

In [26]:
data_obj = pd.merge(data,click_obj)

In [27]:
data_obj

,Stimulus,Participant,Trial,time_step,target,competitor,distractor,trackloss,Item,Condition,Adjektiv_Onset1,Nomen_Onset,adjtime,nountime,Klick on,Klick
0,item121_3,P1,Trial012,3.5,0,0,1,0,121,3,0.575,1.52,-571.5,-1516.5,target,target
1,item121_3,P1,Trial012,7.5,0,0,1,0,121,3,0.575,1.52,-567.5,-1512.5,target,target
2,item121_3,P1,Trial012,11.5,0,0,1,0,121,3,0.575,1.52,-563.5,-1508.5,target,target
3,item121_3,P1,Trial012,15.5,0,0,1,0,121,3,0.575,1.52,-559.5,-1504.5,target,target
4,item121_3,P1,Trial012,19.5,0,0,1,0,121,3,0.575,1.52,-555.5,-1500.5,target,target
5,item121_3,P1,Trial012,23.5,0,0,1,0,121,3,0.575,1.52,-551.5,-1496.5,target,target
6,item121_3,P1,Trial012,27.5,0,0,1,0,121,3,0.575,1.52,-547.5,-1492.5,target,target
7,item121_3,P1,Trial012,31.5,0,0,1,0,121,3,0.575,1.52,-543.5,-1488.5,target,target
8,item121_3,P1,Trial012,35.5,0,0,1,0,121,3,0.575,1.52,-539.5,-1484.5,target,target
9,item121_3,P1,Trial012,39.5,0,0,1,0,121,3,0.575,1.52,-535.5,-1480.5,target,target


In [28]:
index = [ix+1 for ix in range(len(data_obj['Item']))]
data_obj['Row_no'] = index

In [30]:
final_data_obj = data_obj[['Row_no','Stimulus','Participant','Trial','time_step', 'target', \
                           'competitor', 'distractor', 'trackloss', 'Item', 'Condition', 'Adjektiv_Onset1', \
                           'Nomen_Onset','adjtime',  'nountime','Klick']]

In [31]:
final_data_obj.to_csv(r'final_data_obj.txt', header=1, index=None, sep='\t')

## Datenversion, die zwar die Klicks enthält, aber auch alle Datenpunkte aus data

In [25]:
data

,Item,Condition,Trial,Participant,Stimulus,CategoryGroup,CategoryRight,PointRRX,PointRRY,time_step,...,competitor_xmax,competitor_ymax,distractor_xmin,distractor_ymin,distractor_xmax,distractor_ymax,target,competitor,distractor,Row_no
0,121,3,Trial012,P1,item121_3,Eye,Fixation,525.3,920.6,3.5,...,1215,265,363,823,686,1003,0,0,1,1
1,121,3,Trial012,P1,item121_3,Eye,Fixation,524.1,921.7,7.5,...,1215,265,363,823,686,1003,0,0,1,2
2,121,3,Trial012,P1,item121_3,Eye,Fixation,524.3,920.9,11.5,...,1215,265,363,823,686,1003,0,0,1,3
3,121,3,Trial012,P1,item121_3,Eye,Fixation,524.2,918.5,15.5,...,1215,265,363,823,686,1003,0,0,1,4
4,121,3,Trial012,P1,item121_3,Eye,Fixation,522.5,919.3,19.5,...,1215,265,363,823,686,1003,0,0,1,5
5,121,3,Trial012,P1,item121_3,Eye,Fixation,523.5,920.1,23.5,...,1215,265,363,823,686,1003,0,0,1,6
6,121,3,Trial012,P1,item121_3,Eye,Fixation,523.5,919.2,27.5,...,1215,265,363,823,686,1003,0,0,1,7
7,121,3,Trial012,P1,item121_3,Eye,Fixation,525.1,918.3,31.5,...,1215,265,363,823,686,1003,0,0,1,8
8,121,3,Trial012,P1,item121_3,Eye,Fixation,527.1,918.9,35.5,...,1215,265,363,823,686,1003,0,0,1,9
9,121,3,Trial012,P1,item121_3,Eye,Fixation,528.5,918.6,39.5,...,1215,265,363,823,686,1003,0,0,1,10


In [27]:
click

,Participant,Item,Condition,Target,Competitor,Distractor
0,P18,116,3,1,0,0
1,P9,116,3,1,0,0
2,P15,116,3,1,0,0
3,P6,116,3,1,0,0
4,P3,116,3,1,0,0
5,P21,116,3,1,0,0
6,P12,116,3,1,0,0
7,P3,117,1,1,0,0
8,P21,117,1,1,0,0
9,P6,117,1,1,0,0
